<a href="https://colab.research.google.com/github/rb58853/images_RIS-ML-Conv-NLP/blob/main/sam/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### install dependences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install transformers

### tool: create image from a input mask

In [4]:
from PIL import Image

def mask_image(mask, raw_image):
    weigth, heigth = raw_image.size
    new_image = Image.new('RGBA', (weigth, heigth), (0, 0, 0, 0))

    original_pixles = raw_image.load()
    pixels = new_image.load()

    print(new_image)
    new_image.show()

    for i in range (weigth):
        print()
        for j in range (heigth):
            print (mask[i,j], end= " ")
            if mask[i,j]:
                print(" ", end = "")
                pixels[i, j] = original_pixles[i,j]
                # new_image[i][j] = raw_image[i][j]
            else:
                pass
                # new_image[i][j] = 0
    return new_image

### Working with SAM

##### import dependences and model from hugginface

In [5]:
import torch
from PIL import Image
import requests
from transformers import SamModel, SamProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device)
processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

##### select and import image by process

In [6]:
img_url = "/content/drive/MyDrive/Tesis/test_images/4.png"

raw_image = Image.open(img_url).convert("RGB")
input_points = [[[450, 600]]]  # 2D location of a window in the image

##### masks from image

In [7]:
inputs = processor(raw_image, input_points=input_points, return_tensors="pt").to(device)
outputs = model(**inputs)

#Estas son mascaras booleanas True implica que el pixel en cuestion interesa, y False implica que no es de interes, es Fondo de esa capa
masks = processor.image_processor.post_process_masks(
    outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu()
)

In [8]:
for m1 in masks:
  for m2 in m1:
    for m3 in m2:
      print('\n\n-----------------NEW MASK-------------------------')
      mask_image(m3, raw_image).show()





-----------------NEW MASK-------------------------
<PIL.Image.Image image mode=RGBA size=10x10 at 0x7EE519E6CBB0>

tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) 
tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) 
tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) 
tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) 
tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) 
tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) tensor(False) 
tensor(False) tensor(False) tensor(Fa